# 4.9 Task

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

## Importing Data

In [3]:
# main folder
path = r'C:\Users\steve\Documents\11.24 Instacart Basket Analysis'

# customer
cust_df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

# ords_prods
ords_prods = pd.read_pickle(os.path.join(path, '02 data', 'Prepared Data', 'ords_prods_wflags.pkl'))

## Analysis

In [5]:
cust_df.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


**Cleaning Data**

01. Rename Columns

In [10]:
# First Name -> first name
cust_df.rename(columns = {'First Name': 'first name'}, inplace = True)
# Surname -> Last Name
cust_df.rename(columns = {'Surnam': 'last name'}, inplace = True)
# Gender -> gender
cust_df.rename(columns = {'Gender': 'gender'}, inplace = True)
# STATE -> State
cust_df.rename(columns = {'STATE': 'state'}, inplace = True)
# Age -> age
cust_df.rename(columns = {'Age': 'age'}, inplace = True)
# n_dependants -> dependants
cust_df.rename(columns = {'n_dependants': 'dependents'}, inplace = True)

In [11]:
cust_df.head(3)

,user_id,first name,last name,gender,state,age,date_joined,dependents,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568


In [12]:
cust_df.shape

(206209, 10)

**Quality & Consistency Checks**

01. Mixed-Type Data

In [15]:
for col in cust_df.columns.tolist():
    weird = (cust_df[[col]].map(type) != cust_df[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (cust_df[weird]) > 0:
        print(col)

first name


In [23]:
# set first name to type string
cust_df['first name'] = cust_df['first name'].astype('str')

02. Missing Values

In [26]:
cust_df.isnull().sum()

# no missing values

user_id        0
first name     0
last name      0
gender         0
state          0
age            0
date_joined    0
dependents     0
fam_status     0
income         0
dtype: int64

03. Duplicates

In [29]:
df_dups = cust_df[cust_df.duplicated()]
df_dups.head()

# no full duplicates

,user_id,first name,last name,gender,state,age,date_joined,dependents,fam_status,income


## Merge Data

01. Identify shared column

In [33]:
cust_df.head(1)

,user_id,first name,last name,gender,state,age,date_joined,dependents,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665


In [35]:
ords_prods.head(1)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,busiet_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_spent,spending_flag,med_sinceprior,med_since_prior,frequency_label
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,20.5,Non-frequent customer


02. Execute merge

In [38]:
ords_prods_cust = ords_prods.merge(cust_df, on = 'user_id')

In [39]:
ords_prods_cust.head(3)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,frequency_label,first name,last name,gender,state,age,date_joined,dependents,fam_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2539329,1,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,2539329,1,1,2,8,NaN,12427,3,0,Original Beef Jerky,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [40]:
ords_prods_cust.shape

(32399732, 35)

## Export Data

In [42]:
ords_prods_cust.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_cust.pkl'))